# SETUP

In [ ]:
# run IPYNB that imports all the relevant packages and functions
%run ../ms_packages_and_functions.ipynb

In [ ]:
output_dir = pjoin(os.getcwd(), 'processed_data')
data_dir  = pjoin(os.getcwd(), 'raw_data')

In [ ]:
run_path = pjoin(output_dir,'RJ_null-metrics_GAS-003')

if not os.path.exists(run_path):
    os.mkdir(run_path)
    print("created directory: {}".format(run_path))
else:
    print("All files will be saved in: {}".format(run_path))

In [ ]:
fig_dir = pjoin(run_path,'figures')

if not os.path.exists(fig_dir):
    os.mkdir(fig_dir)
    print("created directory: {}".format(fig_dir))
else:
    print("All files will be saved in: {}".format(fig_dir))

# Null paths

In [ ]:
null_folders = [file for file in os.listdir(output_dir) if file.startswith('RJ_nulls-poisson-200ms-MO-0')]
null_filepaths = [pjoin(output_dir,folder,file) for folder in null_folders for file in os.listdir(pjoin(output_dir,folder)) if file.startswith('poisson_FNs_')]

In [ ]:
print(f'{len(null_filepaths)} FOUND IN THE FOLLOWING FILEPATHS: \n{[x[-46:] for x in null_filepaths]}')

# GAS

In [ ]:
run_gas = True

In [ ]:
if run_gas: 
    null_GAS = []
    null_num = 0
    num_processes = 8
    old_i = 0
    for i,FN_path in enumerate(null_filepaths):
        # start_time = time.time()
        try:
            null_FNs = loadPickle(FN_path)
            #############################

            null_alignment_scores = []

            min_trial_length = min([len(z[0]) for y in null_FNs[0] for x in y for z in x])

            all_trials = [[dir,trial] for dir in range(0,len(null_FNs[0]))
                                    for trial in range(0,len(null_FNs[0][dir][0]))]

            FN_pairs = [y for y in itertools.combinations(all_trials,2)]

            for t in range(0,min_trial_length):
                status("> SIM# {} getting alignment scores for time: {}s".format(i,bin2time(t)))
                null_alignment_scores.append([])

                for pair in FN_pairs:
                    score = align_score(null_FNs[0][pair[0][0]][0][pair[0][1]][0][t],null_FNs[0][pair[1][0]][0][pair[1][1]][0][t])
                    null_alignment_scores[t].append([score,pair])

            null_GAS.append(null_alignment_scores)
            
            del null_FNs
            
            ##############################
            # end_time = time.time()
            # print(f'sim # {i}, elapsed (s) {end_time - start_time}')
            null_num += 1

        except Exception as e: 
            print(e)
            end_time = time.time()
            print(f'sim # {i}, broken!')
            # print(f'sim # {i}, broken! elapsed (s) {end_time - start_time}')

        if null_num%5==0 :
            savePickle(pjoin(run_path,f'null_gas-{old_i}-{i}-.pkl'),null_GAS)
            null_GAS = []
            old_i = i
            

# Analyze already generated null GAS values

In [ ]:
null_folders = [file for file in os.listdir(output_dir) if file.startswith('RJ_null-metrics_GAS-') and not file.startswith('RJ_null-metrics_GAS-MO')]
null_filepaths = [pjoin(output_dir,folder,file) for folder in null_folders for file in os.listdir(pjoin(output_dir,folder)) if file.startswith('null_gas') and file.endswith('-.pkl')]
print(f'{len(null_filepaths)} FOUND IN THE FOLLOWING FILEPATHS: \n{[x for x in null_filepaths]}')


In [ ]:
null_gas = loadPickle(null_filepaths[0])
time_gas = [z for z in zip(*null_gas)]
del null_gas

In [ ]:
# load data
data_FNs = loadPickle('processed_data/rj1031209_CenterOut_FN_10msBin_with500buffer_MIonly.pkl')
# data_FNs = loadPickle('processed_data/RJ_CenterOut_FN_10msBin_MOcentered.pkl')


In [ ]:
trials = loadPickle('processed_data/rj031209_trials_by_cutoff.pkl')

In [ ]:
# time_gas = [z for z in zip(*null_gas)]
# del null_gas
cutoffs = [0.5,1,1.5,2,2.5]
for cutoff in cutoffs:
    all_gas = []
    for i,t in enumerate(time_gas):
        gas_by_time = []
        [gas_by_time.append([]) for x in np.arange(5)]
        for gas in zip(*t):
            pair = gas[0][1]

            if (pair[0][1] in trials[cutoff]['trials'][pair[0][0]]) and (pair[1][1] in trials[cutoff]['trials'][pair[1][0]]):
                d_gas = align_score(data_FNs[pair[0][0]][pair[0][1]][i],data_FNs[pair[1][0]][pair[1][1]][i])
                n_gas = [x[0] for x in gas]
                circd = circdistance(8,pair[0][0],pair[1][0])

                gas_by_time[circd].append([pair,d_gas,n_gas])

        all_gas.append(gas_by_time)

    cmap = sns.color_palette('viridis',5)
    
    plt.figure()
    for c in np.arange(5):
        x = [bin2time(x,lastBin=True) for x in np.arange(len(all_gas))]
        temp = [np.mean([np.mean(y[2]) for y in x[c]]) for x in all_gas]
        plt.plot(x,temp,'--',c=cmap[c])

        temp = [np.mean([np.mean(y[1]) for y in x[c]]) for x in all_gas]
        plt.plot(x,temp,'-',c=cmap[c])

    plt.title(cutoff)

    # plt.xlim([0,0.5])
    plt.savefig(pjoin(fig_dir,f'gas-all-{cutoff}.png'))       
    # del data_FNs,time_gas 

In [ ]:
cutoff = 1.5
all_gas = []
for i,t in enumerate(time_gas):
    gas_by_time = []
    [gas_by_time.append([]) for x in np.arange(5)]
    for gas in zip(*t):
        pair = gas[0][1]
        d_gas = align_score(data_FNs[pair[0][0]][pair[0][1]][i],data_FNs[pair[1][0]][pair[1][1]][i])
        n_gas = [x[0] for x in gas]
        circd = circdistance(8,pair[0][0],pair[1][0])

        gas_by_time[circd].append([pair,d_gas,n_gas])

    all_gas.append(gas_by_time)
        
del data_FNs,time_gas 

In [ ]:
savePickle(pjoin(run_path,'temporal_all_gas.pkl'),all_gas)

In [ ]:
# # time_gas = [z for z in zip(*null_gas)]
# # del null_gas

# all_gas = []
# for i,t in enumerate(time_gas):
#     gas_by_time = []
#     [gas_by_time.append([]) for x in np.arange(5)]
#     for gas in zip(*t):
#         pair = gas[0][1]
#         d_gas = align_score(data_FNs[pair[0][0]][pair[0][1]][i],data_FNs[pair[1][0]][pair[1][1]][i])
#         n_gas = [x[0] for x in gas]
#         circd = circdistance(8,pair[0][0],pair[1][0])

#         gas_by_time[circd].append([pair,d_gas,n_gas])

#     all_gas.append(gas_by_time)
        
# del data_FNs,time_gas 

In [ ]:
# cmap = sns.color_palette('viridis',5)
# for c in np.arange(5):
#     x = [bin2time(x,lastBin=True) for x in np.arange(len(all_gas))]
#     temp = [np.mean([np.mean(y[2]) for y in x[c]]) for x in all_gas]
#     plt.plot(x,temp,'--',c=cmap[c])

#     temp = [np.mean([np.mean(y[1]) for y in x[c]]) for x in all_gas]
#     plt.plot(x,temp,'-',c=cmap[c])

# # plt.xlim([0,0.5])
# plt.savefig(pjoin(fig_dir,'gas-all.png'))

# GAS at Movement Onset

In [ ]:
trials = loadPickle('processed_data/rj031209_trials_by_cutoff.pkl')

In [ ]:
data_FNs = loadPickle('processed_data/RJ_CenterOut_FN_10msBin_MOcentered.pkl')


In [ ]:
run_gas_MO = True
if run_gas_MO: 
    null_GAS = []
    null_num = 0
    num_processes = 8
    old_i = 0

    min_trial_length = time2bin(1,lastBin=True,buffer=0) # length of window to plot, bin buffer looks at the time before and after
    binBuffer = min_trial_length/2
    cutoff = 2.5 # we use this since it will get most of the trials and then we can look at less trials later

    for i,FN_path in enumerate(null_filepaths):
        # start_time = time.time()
        try:
            null_FNs = loadPickle(FN_path)
            #############################

            null_alignment_scores = []

            # min_trial_length = min([len(z[0]) for y in null_FNs[0] for x in y for z in x])

            all_trials = [[dir,trial] for dir in range(0,len(null_FNs[0]))
                                    for trial in range(0,len(null_FNs[0][dir][0]))]

            FN_pairs = [y for y in itertools.combinations(all_trials,2)]

            for t in range(0,min_trial_length):
                status("> SIM# {} getting alignment scores for time: {}s".format(i,bin2time(t)))
                null_alignment_scores.append([])
                
                bB = t - (binBuffer)

                for pair in FN_pairs:

                    MOTime_0 = int(time2bin(trials[cutoff]['mv_times'][pair[0][0]][pair[0][1]][0],lastBin=True,buffer=0)+bB)
                    MOTime_1 = int(time2bin(trials[cutoff]['mv_times'][pair[1][0]][pair[1][1]][0],lastBin=True,buffer=0)+bB)

                    score = align_score(null_FNs[0][pair[0][0]][0][pair[0][1]][0][MOTime_0],null_FNs[0][pair[1][0]][0][pair[1][1]][0][MOTime_1])
                    
                    null_alignment_scores[t].append([score,pair])

            null_GAS.append(null_alignment_scores)
            
            del null_FNs
            
            ##############################
            # end_time = time.time()
            # print(f'sim # {i}, elapsed (s) {end_time - start_time}')
            null_num += 1

        except Exception as e: 
            print(e)
            # end_time = time.time()
            print(f'sim # {i}, broken!')
            # print(f'sim # {i}, broken! elapsed (s) {end_time - start_time}')

        if null_num%5==0 :
            savePickle(pjoin(run_path,f'null_gas-MO_{old_i}-{i}-.pkl'),null_GAS)
            null_GAS = []
            old_i = i
            